In [1]:
%%capture _ 
# ^ this just silences warnings
import torch
import mlflow
# import hiddenlayer as HL

from model.collectdata_mdsA import collect_data
from model.collectdata_poca_KDE import collect_data_poca
from model.alt_loss_A import Loss
from model.training import trainNet, select_gpu
from model.utilities import load_full_state, count_parameters, Params, save_to_mlflow

from model.autoencoder_models import UNet
from model.autoencoder_models import UNetPlusPlus

In [2]:
args = Params(
    batch_size=64,
    device = select_gpu(-1),
    epochs=100,
    lr=1e-4,
    experiment_name='CMS Test 220531',
    asymmetry_parameter=0
)

In [6]:
events = 300
## This is used when training with the new KDE
train_loader = collect_data_poca('/data/home/ekauffma/kernel_cms_220526_mod2.h5',
                            batch_size=args['batch_size'],
                            device=args['device'],
                            masking=True, shuffle=True,
                            load_A_and_B=True,
                            load_xy=True,
                            slice = slice(0,300)
                           )

val_loader = collect_data_poca('/data/home/ekauffma/kernel_cms_220526_mod2.h5',
                            batch_size=args['batch_size'],
                            device=args['device'],
                            masking=True, shuffle=True,
                            load_A_and_B=True,
                            load_xy=True,
                            slice = slice(300,366)
                           )

Loading data...
Loaded /data/home/ekauffma/kernel_cms_220526_mod2.h5 in 50.28 s
Constructing 300 event dataset took 0.01165 s
Loading data...
Loaded /data/home/ekauffma/kernel_cms_220526_mod2.h5 in 50.0 s
Constructing 66 event dataset took 0.00597 s


In [7]:
mlflow.tracking.set_tracking_uri('file:/share/lazy/pv-finder_model_repo')
mlflow.set_experiment(args['experiment_name'])

Traceback (most recent call last):
  File "/data/home/ekauffma/.local/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/data/home/ekauffma/.local/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 336, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/data/home/ekauffma/.local/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 175, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/share/lazy/pv-finder_model_repo/ML/meta.yaml' does not exist.
Traceback (most recent call last):
  File "/data/home/ekauffma/.local/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 256, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/data/home/ekauffma/.local/lib/python3

In [8]:
model = UNetPlusPlus().to(args['device'])
#model.to("cuda:0")
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss = Loss(epsilon=1e-5,coefficient=args['asymmetry_parameter'])

parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
#load_full_state(model, optimizer, '/share/lazy/pv-finder_model_repo/24/9a2b98a397eb404497b26ab5eaa091a5/artifacts/train.ipynb')

run_name = 'u-net++ with masking'

# tune kernel based on gpu
#torch.backends.cudnn.benchmark=True
train_iter = enumerate(trainNet(model, optimizer, loss, train_loader, val_loader, args['epochs'], notebook=True))
with mlflow.start_run(run_name = run_name) as run:
    mlflow.log_artifact('train.ipynb')
    for i, result in train_iter:
        print(result.cost)
        torch.save(model, 'run_stats.pyt')
        mlflow.log_artifact('run_stats.pyt')

        save_to_mlflow({
            'Metric: Training loss':result.cost,
            'Metric: Validation loss':result.val,
            'Metric: Efficiency':result.eff_val.eff_rate,
            'Metric: False positive rate':result.eff_val.fp_rate,
            'Param: Parameters':parameters,
            'Param: Events':events,
            'Param: Asymmetry':args['asymmetry_parameter'],
            'Param: Epochs':args['epochs'],
        }, step=i)


Number of batches: train = 5, val = 2


Epoch 0: train=594.653, val=326.401, took 100.09 s
  Validation Found 16 of 1658, added 169 (eff 0.97%) (2.52 FP/event)
594.6525817871094


Epoch 1: train=584.948, val=322.175, took 96.054 s
  Validation Found 25 of 1658, added 279 (eff 1.51%) (4.16 FP/event)
584.9484130859375


Epoch 2: train=573.8, val=316.428, took 98.723 s
  Validation Found 25 of 1658, added 352 (eff 1.51%) (5.25 FP/event)
573.8001403808594


Epoch 3: train=557.921, val=307.553, took 97.298 s
  Validation Found 34 of 1658, added 437 (eff 2.05%) (6.52 FP/event)
557.9210815429688


...

Epoch 95: train=9.98546, val=5.58893, took 93.665 s
  Validation Found 879 of 1658, added 257 (eff 53.02%) (3.84 FP/event)
9.985457706451417


Epoch 96: train=10.006, val=5.5897, took 93.382 s
  Validation Found 876 of 1658, added 255 (eff 52.83%) (3.81 FP/event)
10.005967903137208


Epoch 97: train=9.97796, val=5.60146, took 94.581 s
  Validation Found 878 of 1658, added 263 (eff 52.96%) (3.93 FP/event)
9.977959823608398


Epoch 98: train=9.9853, val=5.57963, took 92.379 s
  Validation Found 877 of 1658, added 263 (eff 52.90%) (3.93 FP/event)
9.985304164886475


Epoch 99: train=9.99491, val=5.57991, took 96.393 s
  Validation Found 878 of 1658, added 261 (eff 52.96%) (3.9 FP/event)
9.994914245605468

